In [5]:
# 1. Bibliotheken importieren
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from model import HockeyActionModel   # Modell aus model.py
from dataloader import HockeyDataset  # DataLoader aus dataloader.py
import os 

In [6]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


False


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# 2. Device auswählen
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Benutze Gerät: {device}")

Benutze Gerät: cpu


In [ ]:
def custom_collate(batch):
    videos = torch.stack([item[0] for item in batch])  # [B, T, 3, 224, 224]
    labels = torch.tensor([item[1] for item in batch])  # [B]
    return videos, labels


In [ ]:
x, y = dataset[0]
print(type(x))  # Muss <class 'torch.Tensor'> sein
print(x.shape)  # Muss z. B. torch.Size([100, 3, 224, 224]) sein


<class 'list'>


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
# 3. Hyperparameter
batch_size = 2
num_epochs = 10
learning_rate = 0.001
num_classes = 4
frames_per_clip = 100 # die letzen 100 Frames pro clip werden für die Vorhersage verwendet

In [ ]:
# 4. Modell instanziieren
# Bildtransformationen
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Hyperparameter
frames_per_clip = 100
batch_size = 2

# Dataset instanziieren
dataset = HockeyDataset(
    labels_csv="labels.csv",
    frames_root="frames",
    transform=transform,
    frames_per_clip=frames_per_clip
)

train_loader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    num_workers=0,  # statt 2
    collate_fn=custom_collate
)




In [ ]:
# 5. Modell laden
model = HockeyActionModel(num_classes=num_classes).to(device)

c:\Users\hp\anaconda3\envs\picture\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\hp\anaconda3\envs\picture\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# 6. Loss und Optimizer definieren
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Optional, falls du String-Labels hast:
label_map = {'Check': 0, 'Neutral': 1, 'Schuss': 2, 'Tor': 3}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for frames, labels in train_loader:  # 👈 HIER steht train_loader
        frames = frames.to(device)
        labels = labels.to(device)

        outputs = model(frames)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Acc: {100*correct/total:.2f}%")



TypeError: expected Tensor as element 0 in argument 0, but got list

In [ ]:
# 8. Modell speichern
os.makedirs('models', exist_ok=True)
torch.save(model.state_dict(), 'models/hockey_action_model.pth')
print("✅ Modell erfolgreich gespeichert!")